# 1. LeNet

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
device = torch.device('cuda:0')

In [3]:
# 트레이닝 데이터셋을 다운로드한다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 테스트 데이터셋을 다운로드한다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # Concatenating along the channel dimension (dim=1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)



class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [5]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [6]:
print(UNet(1, 10))

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()  # Set model to training mode
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        # Simulate segmentation by repeating the label across spatial dimensions
        # Make `y` a 3D tensor of shape [batch_size, height, width]
        y = y.view(-1, 1, 1)  # Reshape to [batch_size, 1, 1]
        y = y.expand(-1, X.shape[2], X.shape[3])  # Repeat for spatial dimensions

        # Forward pass: get model prediction
        pred = model(X)

        # Compute the loss
        loss = loss_fn(pred, y)  

        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters

        if batch % 100 == 0:
            loss_value, current = loss.item(), batch * len(X)
            print(f'loss: {loss_value:.4f}, [{current:>5d}/{size:>5d}]')


def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()  # Set the model to evaluation mode
    
    with torch.no_grad():  # Disable gradient calculation
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            # Simulate segmentation by repeating the label across spatial dimensions
            y = y.view(-1, 1, 1)  # Reshape to [batch_size, 1, 1]
            y = y.expand(-1, X.shape[2], X.shape[3])  # Repeat for spatial dimensions

            pred = model(X)  # Forward pass
            test_loss += loss_fn(pred, y).item()  # Accumulate the loss

            # Calculate the number of correct predictions
            pred_labels = pred.argmax(1)  # Get the predicted class labels
            correct += (pred_labels == y).type(torch.float).sum().item()  # Sum the correct predictions
    
    # Compute the average loss over all batches
    test_loss /= num_batches

    # Compute accuracy as the percentage of correctly classified pixels
    correct /= (size * X.shape[2] * X.shape[3])  # Normalize by the total number of pixels
    accuracy = 100 * correct  # Convert to percentage

    print(f'Test Error: \n Accuracy: {accuracy:>0.1f}% Average Loss: {test_loss:>8f}\n')

In [8]:
def run(device):
    #device = 'cuda:0' if torch.cuda.is_available() else 'cpu'   
    #device = 'cpu'
    print(f"사용할 장치: {device}")

    model = UNet(1, 10).to(device)

    learning_rate = 0.001
    batch_size = 64
    epochs = 10


    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f'Epoch {t+1}\n--------------------------------------')
        train_loop(train_dataloader, model, loss_fn, optimizer, device)
        test_loop(train_dataloader, model, loss_fn, device)
    print("Done!")

In [9]:
run(device)

사용할 장치: cuda:0
Epoch 1
--------------------------------------
loss: 2.4317, [    0/60000]
loss: 2.1884, [ 6400/60000]
loss: 1.8215, [12800/60000]
loss: 1.6845, [19200/60000]
loss: 1.5242, [25600/60000]
loss: 1.3672, [32000/60000]
loss: 1.1939, [38400/60000]
loss: 1.1345, [44800/60000]
loss: 1.0470, [51200/60000]
loss: 1.0282, [57600/60000]
Test Error: 
 Accuracy: 78.9% Average Loss: 0.895083

Epoch 2
--------------------------------------
loss: 0.9663, [    0/60000]
loss: 0.9392, [ 6400/60000]
loss: 0.7711, [12800/60000]
loss: 0.8738, [19200/60000]
loss: 0.7505, [25600/60000]
loss: 0.7683, [32000/60000]
loss: 0.6764, [38400/60000]
loss: 0.7847, [44800/60000]
loss: 0.6481, [51200/60000]
loss: 0.6796, [57600/60000]
Test Error: 
 Accuracy: 86.5% Average Loss: 0.559708

Epoch 3
--------------------------------------
loss: 0.6181, [    0/60000]
loss: 0.6604, [ 6400/60000]
loss: 0.5035, [12800/60000]
loss: 0.6266, [19200/60000]
loss: 0.5443, [25600/60000]
loss: 0.5486, [32000/60000]
loss: 0.